In [ ]:
import numpy as np

# Function to calculate term innovation
def calculate_term_innovation(x, p, v, delta):
    n = len(x)
    m = np.zeros(n)
    s = np.zeros(n)
    c = np.zeros(n)
    d = np.zeros(n)
    a = np.zeros(n)
    g = np.zeros(n)
    h = np.zeros(n)

    for i in range(p, n):
        # Calculate mean and standard deviation
        m[i] = np.mean(x[i-p:i])
        s[i] = np.std(x[i-p:i])
        
        # Calculate c, d, a, g, h
        exp_term = np.exp(v * (x[i] - m[i]) / s[i])
        c[i] = 1 / (1 + exp_term)
        d[i] = delta * (1 - c[i]) * (x[i] - m[i])
        a[i] = m[i] + d[i]
        g[i] = (v * delta * c[i] * s[i] * s[i]) / ((1 + exp_term) ** 2)
        h[i] = 1 - (v * delta * c[i] * (x[i] - m[i])) / (s[i] * (1 + exp_term))

    return {'m': m, 's': s, 'c': c, 'd': d, 'a': a, 'g': g, 'h': h}

# Function to calculate window comparison
def calculate_window_comparison(x, p, v, delta):
    term_innovations = []
    n = len(x)

    for i in range(p, n):
        # Calculate term innovations for each window
        term_innovations.append(calculate_term_innovation(x[0:i+1], p, v, delta))

    # Calculate mean and standard deviation of term innovations
    term_innovations_m = np.zeros(n)
    term_innovations_s = np.zeros(n)

    for i in range(p, n):
        term_innovations_m[i] = np.mean([term_innovations[j]['a'][i] for j in range(i-p,i)])
        term_innovations_s[i] = np.std([term_innovations[j]['a'][i] for j in range(i-p,i)])
    
    # Calculate T, U and V
    T = np.zeros(n)
    U = np.zeros(n)
    V = np.zeros(n)
    
    for i in range(p, n):
        T[i] = (term_innovations_m[i] - np.mean(x[0:i+1])) / np.std(x[0:i+1])
        U[i] = (term_innovations_s[i] / np.std(x[0:i+1])) ** 2
        V[i] = (np.mean([term_innovations[j]['g'][i] for j in range(i-p,i)]) * (1 - U[i])) + (np.mean([term_innovations[j]['h'][i] for j in range(i-p,i)]) * U[i])
    
    return {'T': T, 'U': U, 'V': V}
